In [1]:
import os
import torch
import numpy as np
import faiss
from pathlib import Path
from typing import Dict, List, Tuple, Optional
import pickle
from tqdm import tqdm


In [2]:
class FaissDatabase:
    """
    Класс для работы с FAISS базой данных эмбеддингов.
    Поддерживает создание индекса, добавление эмбеддингов и поиск ближайших соседей.
    """
    
    def __init__(self, dimension: int = 512, index_type: str = "flat"):
        """
        Инициализация FAISS базы данных.
        
        Args:
            dimension: Размерность эмбеддингов (по умолчанию 512)
            index_type: Тип индекса ("flat", "ivf", "hnsw")
        """
        self.dimension = dimension
        self.index_type = index_type
        self.index = None
        self.metadata = {}  # Словарь для хранения метаданных по индексам
        self.id_to_path = {}  # Маппинг ID в базе к пути файла
        self.path_to_id = {}  # Обратный маппинг
        self.next_id = 0
        
    def _create_index(self, n_vectors: int = 0):
        """Создает FAISS индекс в зависимости от типа."""
        if self.index_type == "flat":
            # Точный поиск, медленный для больших баз
            self.index = faiss.IndexFlatIP(self.dimension)  # Inner Product (cosine similarity)
        elif self.index_type == "ivf":
            # Приближенный поиск, быстрый для больших баз
            quantizer = faiss.IndexFlatIP(self.dimension)
            nlist = min(100, max(1, n_vectors // 100))  # Количество кластеров
            self.index = faiss.IndexIVFFlat(quantizer, self.dimension, nlist)
        elif self.index_type == "hnsw":
            # Графовый индекс, хороший баланс скорости и точности
            self.index = faiss.IndexHNSWFlat(self.dimension, 32)
        else:
            raise ValueError(f"Неподдерживаемый тип индекса: {self.index_type}")
    
    def add_embeddings_from_files(self, 
                                 embeddings_dir: str, 
                                 metadata: Dict[str, any] = None,
                                 file_pattern: str = "*.pt") -> int:
        """
        Добавляет эмбеддинги из файлов в базу данных.
        
        Args:
            embeddings_dir: Путь к папке с .pt файлами эмбеддингов
            metadata: Словарь с метаданными (опционально)
            file_pattern: Паттерн для поиска файлов (по умолчанию "*.pt")
            
        Returns:
            Количество добавленных эмбеддингов
        """
        embeddings_dir = Path(embeddings_dir)
        if not embeddings_dir.exists():
            raise ValueError(f"Папка {embeddings_dir} не существует")
        
        # Находим все .pt файлы
        pt_files = list(embeddings_dir.rglob(file_pattern))
        if not pt_files:
            print(f"Не найдено файлов с паттерном {file_pattern} в {embeddings_dir}")
            return 0
        
        print(f"Найдено {len(pt_files)} файлов эмбеддингов")
        
        # Загружаем все эмбеддинги
        embeddings = []
        file_paths = []
        
        for pt_file in tqdm(pt_files, desc="Загрузка эмбеддингов"):
            try:
                embedding = torch.load(pt_file, map_location='cpu')
                
                # Проверяем размерность
                if embedding.dim() > 1:
                    embedding = embedding.flatten()
                
                if embedding.shape[0] != self.dimension:
                    print(f"Пропускаем {pt_file}: неверная размерность {embedding.shape[0]} (ожидается {self.dimension})")
                    continue
                
                embeddings.append(embedding.numpy())
                file_paths.append(str(pt_file.name))
                
            except Exception as e:
                print(f"Ошибка при загрузке {pt_file}: {e}")
                continue
        
        if not embeddings:
            print("Не удалось загрузить ни одного эмбеддинга")
            return 0
        
        # Создаем индекс если его еще нет
        if self.index is None:
            self._create_index(len(embeddings))
        
        # Нормализуем эмбеддинги для cosine similarity
        embeddings = np.array(embeddings, dtype=np.float32)
        faiss.normalize_L2(embeddings)
        
        # Добавляем в индекс
        if self.index_type == "ivf" and not self.index.is_trained:
            print("Обучение IVF индекса...")
            self.index.train(embeddings)
        
        self.index.add(embeddings)
        
        # Сохраняем метаданные
        for i, file_path in enumerate(file_paths):
            current_id = self.next_id + i
            self.id_to_path[current_id] = file_path
            self.path_to_id[file_path] = current_id
            
            if metadata:
                self.metadata[current_id] = metadata.get(file_path, {})
            else:
                self.metadata[current_id] = {"file_path": file_path}
        
        self.next_id += len(embeddings)
        
        print(f"Добавлено {len(embeddings)} эмбеддингов в базу данных")
        return len(embeddings)
    
    def add_single_embedding(self, 
                           embedding: np.ndarray, 
                           file_path: str, 
                           metadata: Dict[str, any] = None) -> int:
        """
        Добавляет один эмбеддинг в базу данных.
        
        Args:
            embedding: Эмбеддинг как numpy массив
            file_path: Путь к файлу эмбеддинга
            metadata: Метаданные для эмбеддинга
            
        Returns:
            ID добавленного эмбеддинга
        """
        if embedding.shape[0] != self.dimension:
            raise ValueError(f"Неверная размерность эмбеддинга: {embedding.shape[0]} (ожидается {self.dimension})")
        
        # Создаем индекс если его еще нет
        if self.index is None:
            self._create_index()
        
        # Нормализуем эмбеддинг
        embedding = embedding.astype(np.float32).reshape(1, -1)
        faiss.normalize_L2(embedding)
        
        # Добавляем в индекс
        if self.index_type == "ivf" and not self.index.is_trained:
            self.index.train(embedding)
        
        self.index.add(embedding)
        
        # Сохраняем метаданные
        current_id = self.next_id
        self.id_to_path[current_id] = Path(file_path).name
        self.path_to_id[file_path] = current_id
        self.metadata[current_id] = metadata or {"file_path": Path(file_path).name}
        
        self.next_id += 1
        return current_id
    
    def search(self, 
               query_embedding: np.ndarray, 
               k: int = 5, 
               return_metadata: bool = True) -> List[Dict]:
        """
        Ищет k ближайших соседей для запроса.
        
        Args:
            query_embedding: Эмбеддинг запроса
            k: Количество ближайших соседей
            return_metadata: Возвращать ли метаданные
            
        Returns:
            Список словарей с результатами поиска
        """
        if self.index is None or self.index.ntotal == 0:
            print("База данных пуста")
            return []
        
        if query_embedding.shape[0] != self.dimension:
            raise ValueError(f"Неверная размерность запроса: {query_embedding.shape[0]} (ожидается {self.dimension})")
        
        # Нормализуем запрос
        query_embedding = query_embedding.astype(np.float32).reshape(1, -1)
        faiss.normalize_L2(query_embedding)
        
        # Выполняем поиск
        distances, indices = self.index.search(query_embedding, min(k, self.index.ntotal))
        
        results = []
        for i, (distance, idx) in enumerate(zip(distances[0], indices[0])):
            if idx == -1:  # faiss возвращает -1 для несуществующих индексов
                continue
                
            result = {
                "rank": i + 1,
                "distance": float(distance),
                "similarity": float(distance),  # Для cosine similarity distance = similarity
                "id": int(idx)
            }
            
            if return_metadata:
                result["file_path"] = self.id_to_path.get(idx, "unknown")
                result["metadata"] = self.metadata.get(idx, {})
            
            results.append(result)
        
        return results
    
    def get_stats(self) -> Dict:
        """Возвращает статистику базы данных."""
        if self.index is None:
            return {"total_vectors": 0, "dimension": self.dimension, "index_type": self.index_type}
        
        return {
            "total_vectors": self.index.ntotal,
            "dimension": self.dimension,
            "index_type": self.index_type,
            "is_trained": getattr(self.index, 'is_trained', True)
        }
    
    def save_index(self, filepath: str):
        """Сохраняет индекс и метаданные в папку."""
        if self.index is None:
            print("Нет данных для сохранения")
            return
        
        filepath = Path(filepath)
        filepath.parent.mkdir(parents=True, exist_ok=True)
        
        # Сохраняем индекс
        index_file = filepath / 'index.faiss'
        faiss.write_index(self.index, str(index_file))
        
        # Сохраняем метаданные
        metadata_file = filepath / 'metadata.pkl'
        with open(metadata_file, 'wb') as f:
            pickle.dump({
                'metadata': self.metadata,
                'id_to_path': self.id_to_path,
                'path_to_id': self.path_to_id,
                'next_id': self.next_id,
                'dimension': self.dimension,
                'index_type': self.index_type
            }, f)
        
        print(f"Индекс сохранен: {index_file}")
        print(f"Метаданные сохранены: {metadata_file}")
    
    def load_index(self, filepath: str):
        """Загружает индекс и метаданные из папки."""
        filepath = Path(filepath)
        
        # Загружаем индекс
        index_file = filepath / 'index.faiss'
        if not index_file.exists():
            raise FileNotFoundError(f"Файл индекса не найден: {index_file}")
        
        self.index = faiss.read_index(str(index_file))
        
        # Загружаем метаданные
        metadata_file = filepath / 'metadata.pkl'
        if not metadata_file.exists():
            raise FileNotFoundError(f"Файл метаданных не найден: {metadata_file}")
        
        with open(metadata_file, 'rb') as f:
            data = pickle.load(f)
            self.metadata = data['metadata']
            self.id_to_path = data['id_to_path']
            self.path_to_id = data['path_to_id']
            self.next_id = data['next_id']
            self.dimension = data['dimension']
            self.index_type = data['index_type']
        
        print(f"Индекс загружен: {index_file}")
        print(f"Метаданные загружены: {metadata_file}")
        print(f"Загружено {self.index.ntotal} векторов")


In [3]:
import pandas as pd

metadata = pd.read_csv("/home/borntowarn/projects/chest-diseases/training/data/CT-RATE/dataset/multi_abnormality_labels/train_predicted_labels.csv")
metadata = {
    row['VolumeName'].replace('.gz', '.pt'): list(row.iloc[1:]) + [0, 0]
    for i, row in metadata.iterrows()
}



In [4]:
val_metadata = pd.read_csv("/home/borntowarn/projects/chest-diseases/training/data/CT-RATE/dataset/multi_abnormality_labels/valid_predicted_labels.csv")
metadata.update({
    row['VolumeName'].replace('.gz', '.pt'): list(row.iloc[1:]) + [0, 0]
    for i, row in val_metadata.iterrows()
})


In [3]:
db = FaissDatabase(dimension=512, index_type="flat")

In [4]:
db.load_index('/home/borntowarn/projects/chest-diseases/training/weights/faiss_database/train')

Индекс загружен: /home/borntowarn/projects/chest-diseases/training/weights/faiss_database/train/index.faiss
Метаданные загружены: /home/borntowarn/projects/chest-diseases/training/weights/faiss_database/train/metadata.pkl
Загружено 48825 векторов


In [16]:
# Пример использования FaissDatabase

embeddings_dir = "/home/borntowarn/projects/chest-diseases/training/data/CT-RATE/dataset/valid_fixed_embeds_not_normalized_lipro"

num_added = db.add_embeddings_from_files(embeddings_dir, metadata)
print(f"Добавлено {num_added} эмбеддингов")

Найдено 3039 файлов эмбеддингов


Загрузка эмбеддингов: 100%|██████████| 3039/3039 [00:02<00:00, 1275.14it/s]


Добавлено 3039 эмбеддингов в базу данных
Добавлено 3039 эмбеддингов


In [18]:
# Добавляем эмбеддинги из MosMed

mosmed_embeds_path = '/home/borntowarn/projects/chest-diseases/training/notebooks/preprocessed_mosmed/train_data.pt'
mosmed_labels_path = '/home/borntowarn/projects/chest-diseases/training/notebooks/preprocessed_mosmed/train_labels.pt'

mosmed_embeds1 = torch.load(mosmed_embeds_path)  # shape: (N, D)
mosmed_labels1 = torch.load(mosmed_labels_path)  # shape: (N, ...)

mosmed_embeds_path = '/home/borntowarn/projects/chest-diseases/training/notebooks/preprocessed_mosmed/test_data.pt'
mosmed_labels_path = '/home/borntowarn/projects/chest-diseases/training/notebooks/preprocessed_mosmed/test_labels.pt'

mosmed_embeds2 = torch.load(mosmed_embeds_path)  # shape: (N, D)
mosmed_labels2 = torch.load(mosmed_labels_path)  # shape: (N, ...)

mosmed_embeds_path = '/home/borntowarn/projects/chest-diseases/training/notebooks/preprocessed_mosmed/val_data.pt'
mosmed_labels_path = '/home/borntowarn/projects/chest-diseases/training/notebooks/preprocessed_mosmed/val_labels.pt'

mosmed_embeds3 = torch.load(mosmed_embeds_path)  # shape: (N, D)
mosmed_labels3 = torch.load(mosmed_labels_path)  # shape: (N, ...)

mosmed_embeds = torch.cat([mosmed_embeds1, mosmed_embeds2, mosmed_embeds3], dim=0)
mosmed_labels = torch.cat([mosmed_labels1, mosmed_labels2, mosmed_labels3], dim=0)

# Преобразуем лейблы к спискам, если это тензор
if isinstance(mosmed_labels, torch.Tensor):
    mosmed_labels = mosmed_labels.tolist()

In [21]:
# Добавляем эмбеддинги и лейблы в базу
added = 0
for i in range(len(mosmed_embeds)):
    emb = mosmed_embeds[i]
    label = mosmed_labels[i]
    db.add_single_embedding(emb.numpy(), file_path=f'mosmed_{i}.pt', metadata=label)
    added += 1

print(f"Добавлено {added} эмбеддингов из MosMed")

Добавлено 1866 эмбеддингов из MosMed


In [23]:
db.save_index('/home/borntowarn/projects/chest-diseases/training/notebooks/faiss_database_all')

Индекс сохранен: /home/borntowarn/projects/chest-diseases/training/notebooks/faiss_database_all.faiss
Метаданные сохранены: /home/borntowarn/projects/chest-diseases/training/notebooks/faiss_database_all.metadata.pkl


In [22]:
# 3. Получаем статистику
stats = db.get_stats()
print(f"Статистика базы данных: {stats}")

Статистика базы данных: {'total_vectors': 52051, 'dimension': 512, 'index_type': 'flat', 'is_trained': True}


# CT - RATE

In [51]:
import pandas as pd
results = {}
k = 30

metadata = pd.read_csv("/home/borntowarn/projects/chest-diseases/training/data/CT-RATE/dataset/multi_abnormality_labels/valid_predicted_labels.csv")
y_true = [int(any(list(row.iloc[1:]))) for i, row in metadata.iterrows()]

for file in tqdm(Path('/home/borntowarn/projects/chest-diseases/training/data/CT-RATE/dataset/valid_fixed_embeds_not_normalized_lipro').rglob('*.pt')):
    random_embedding = torch.load(file).flatten().numpy()
    results[file.name.replace('.pt', '.gz')] = db.search(random_embedding, k=30)

3039it [01:01, 49.47it/s]


In [52]:
# Для вероятности принадлежности к классу 1 можно взять отношение total_true / k для каждого результата
y_proba = []
for i, row in metadata.iterrows():
    result = results[row['VolumeName']]
    total_true = sum(1 for r in result if any(r['metadata']))
    proba = total_true / k
    y_proba.append(proba)

# Вероятность принадлежности к классу 0 — это 1 - proba
y_proba_0 = [1 - p for p in y_proba]
y_proba_1 = y_proba


In [57]:
from sklearn.metrics import roc_auc_score, precision_recall_curve, f1_score, accuracy_score, precision_score, recall_score, confusion_matrix, auc
print('roc_auc_score', round(roc_auc_score(y_true, y_proba), 3))
y, x, t = precision_recall_curve(y_true, y_proba)
print('pr_auc_score', round(auc(x, y), 3))

y_pred = [1 if p >= 0.7 else 0 for p in y_proba]

print('f1_score', round(f1_score(y_true, y_pred), 3))
print('accuracy_score', round(accuracy_score(y_true, y_pred), 3))
print('precision_score', round(precision_score(y_true, y_pred), 3))
print('recall_score', round(recall_score(y_true, y_pred), 3))

# Подсчет sensitivity (чувствительность) и specificity (специфичность)
cm = confusion_matrix(y_true, y_pred)
tn, fp, fn, tp = cm.ravel()
sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
print('sensitivity (recall)', round(sensitivity, 3))
print('specificity', round(specificity, 3))


roc_auc_score 0.776
pr_auc_score 0.964
f1_score 0.908
accuracy_score 0.839
precision_score 0.915
recall_score 0.902
sensitivity (recall) 0.902
specificity 0.347


# MosMed

In [48]:
results = []
k = 30

embs = torch.load('/home/borntowarn/projects/chest-diseases/training/notebooks/preprocessed_mosmed/test_data.pt')
lbls = torch.load('/home/borntowarn/projects/chest-diseases/training/notebooks/preprocessed_mosmed/test_labels.pt')
y_true = [int(any(lbl)) for lbl in lbls.tolist()]

for emb in tqdm(embs):
    results.append(db.search(emb.numpy(), k=k))

  0%|          | 0/94 [00:00<?, ?it/s]

100%|██████████| 94/94 [00:01<00:00, 55.84it/s]


In [49]:
# Для вероятности принадлежности к классу 1 можно взять отношение total_true / k для каждого результата
y_proba = []
for result in results:
    total_true = sum(1 for r in result if any(r['metadata']))
    proba = total_true / k
    y_proba.append(proba)

# Вероятность принадлежности к классу 0 — это 1 - proba
y_proba_0 = [1 - p for p in y_proba]
y_proba_1 = y_proba


In [50]:
from sklearn.metrics import roc_auc_score, precision_recall_curve, f1_score, accuracy_score, precision_score, recall_score, confusion_matrix
print('roc_auc_score', round(roc_auc_score(y_true, y_proba), 3))
y, x, t = precision_recall_curve(y_true, y_proba)
print('pr_auc_score', round(auc(x, y), 3))

y_pred = [1 if p >= 0.8 else 0 for p in y_proba]

print('f1_score', round(f1_score(y_true, y_pred), 3))
print('accuracy_score', round(accuracy_score(y_true, y_pred), 3))
print('precision_score', round(precision_score(y_true, y_pred), 3))
print('recall_score', round(recall_score(y_true, y_pred), 3))

# Подсчет sensitivity (чувствительность) и specificity (специфичность)
cm = confusion_matrix(y_true, y_pred)
tn, fp, fn, tp = cm.ravel()
sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
print('sensitivity (recall)', round(sensitivity, 3))
print('specificity', round(specificity, 3))


roc_auc_score 0.713
pr_auc_score 0.9
f1_score 0.8
accuracy_score 0.702
precision_score 0.836
recall_score 0.767
sensitivity (recall) 0.767
specificity 0.476
